# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [38]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import IndexToString, StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [4]:
spark = SparkSession.builder.appName('lr_hunday').getOrCreate()

In [14]:
df = spark.read.csv('gs://hkanjih-spark-udemy/material/Spark_for_Machine_Learning/Linear_Regression/cruise_ship_info.csv',inferSchema=True, header=True)

In [15]:
df.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|Ship_name|Cruise_line|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|      158|        158|               158|               158|              158|              158|               158|              158|              158|
|   mean| Infinity|       null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev|      NaN|       null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|Adventure|    Azamara|   

In [12]:
df.printSchema()
## Cruise_line is important

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [16]:
df.head(1)

[Row(Ship_name=u'Journey', Cruise_line=u'Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)]

In [19]:
df.groupBy(df['Cruise_line']).count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [27]:
df.createOrReplaceGlobalTempView("ship")
sqlDF = spark.sql('Select Cruise_line, Crew from global_temp.ship group by Cruise_line, Crew ').show()

+----------------+-----+
|     Cruise_line| Crew|
+----------------+-----+
|Holland_American| 5.57|
|Holland_American|  5.3|
|             MSC| 2.97|
|          Cunard| 9.21|
|            Star| 0.59|
|         Crystal| 5.45|
|Holland_American| 6.12|
|Holland_American| 5.31|
|        Carnival| 10.3|
|       Silversea| 1.97|
|       Celebrity|  0.6|
|Holland_American| 6.44|
|        Carnival|10.29|
|Holland_American| 5.88|
|        Carnival|  6.7|
|             MSC|  7.0|
|       Celebrity| 8.58|
|             MSC|  7.6|
|       Celebrity|  6.7|
|             MSC|13.13|
+----------------+-----+
only showing top 20 rows



In [30]:
indexer = StringIndexer(inputCol="Cruise_line", outputCol="cruiseLineIndex")
modelIndex = indexer.fit(df)
indexed = modelIndex.transform(df)

In [32]:
indexed.head(1)

[Row(Ship_name=u'Journey', Cruise_line=u'Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruiseLineIndex=16.0)]

In [36]:
indexed.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- cruiseLineIndex: double (nullable = true)



In [41]:
assembler = VectorAssembler(inputCols=[ 'Tonnage',
 'passengers',
 'length',
 'cabins','passenger_density','cruiseLineIndex'],outputCol='features')

In [42]:
output = assembler.transform(indexed)

In [44]:
output.head(2)

[Row(Ship_name=u'Journey', Cruise_line=u'Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruiseLineIndex=16.0, features=DenseVector([30.277, 6.94, 5.94, 3.55, 42.64, 16.0])),
 Row(Ship_name=u'Quest', Cruise_line=u'Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruiseLineIndex=16.0, features=DenseVector([30.277, 6.94, 5.94, 3.55, 42.64, 16.0]))]

In [45]:
final_data = output.select('features','crew' )

In [46]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[30.2769999999999...|3.55|
|[30.2769999999999...|3.55|
|[47.262,14.86,7.2...| 6.7|
|[110.0,29.74,9.53...|19.1|
|[101.353,26.42,8....|10.0|
|[70.367,20.52,8.5...| 9.2|
|[70.367,20.52,8.5...| 9.2|
|[70.367,20.56,8.5...| 9.2|
|[70.367,20.52,8.5...| 9.2|
|[110.238999999999...|11.5|
|[110.0,29.74,9.51...|11.6|
|[46.052,14.52,7.2...| 6.6|
|[70.367,20.52,8.5...| 9.2|
|[70.367,20.52,8.5...| 9.2|
|[86.0,21.24,9.63,...| 9.3|
|[110.0,29.74,9.51...|11.6|
|[88.5,21.24,9.63,...|10.3|
|[70.367,20.52,8.5...| 9.2|
|[88.5,21.24,9.63,...| 9.3|
|[70.367,20.52,8.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [47]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [48]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               101|
|   mean| 8.022970297029705|
| stddev|3.4679136507573114|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [49]:
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                57|
|   mean| 7.388771929824561|
| stddev|3.5601615483888605|
|    min|              0.59|
|    max|              13.6|
+-------+------------------+



In [54]:
lr = LinearRegression(labelCol='crew')

In [55]:
lr_model = lr.fit(train_data)

In [56]:
test_results = lr_model.evaluate(test_data)

In [57]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  0.4957232933217307|
| 0.41398795377104036|
|0.042055880063509354|
|  0.6234544732992109|
|  1.2009334369484286|
| -0.0789788321077931|
|0.009051081496254554|
| 0.13374236793776584|
| 0.10034301492526154|
|   0.267065296093584|
|   0.267065296093584|
|  0.4376489940780357|
| 0.21541732506024225|
| 0.14690335422633716|
|-0.01323459905658...|
|-0.05304104887741268|
| -0.3993014475384884|
|-0.07224797641175051|
|  1.9171752261872843|
|  0.4252977997746399|
+--------------------+
only showing top 20 rows



In [58]:
test_results.rootMeanSquaredError

0.6962999391674718

In [59]:
test_results.r2

0.9610650041635542

In [60]:
unlabeled_data = test_data.select('features')

In [61]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[2.329,0.94,2.96,...|
|[3.341,0.66,2.8,0...|
|[10.0,2.08,4.4,1....|
|[12.5,3.94,4.36,0...|
|[19.093,8.0,5.37,...|
|[19.2,3.2,5.13,1....|
|[25.0,3.88,5.97,1...|
|[25.0,7.76,6.22,3...|
|[28.43,8.08,6.16,...|
|[30.2769999999999...|
|[30.2769999999999...|
|[30.2769999999999...|
|[33.0,4.9,5.6,2.4...|
|[34.25,10.52,6.15...|
|[38.0,10.56,5.67,...|
|[42.0,15.04,7.08,...|
|[44.348,12.0,7.54...|
|[45.0,11.78,7.54,...|
|[46.0,7.0,6.7,1.8...|
|[47.262,14.86,7.2...|
+--------------------+
only showing top 20 rows



In [62]:
predictions= lr_model.transform(unlabeled_data)

In [63]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|[2.329,0.94,2.96,...|0.10427670667826927|
|[3.341,0.66,2.8,0...| 0.1760120462289596|
|[10.0,2.08,4.4,1....| 1.5579441199364907|
|[12.5,3.94,4.36,0...| 0.8365455267007891|
|[19.093,8.0,5.37,...| 3.4990665630515716|
|[19.2,3.2,5.13,1....|  2.188978832107793|
|[25.0,3.88,5.97,1...| 2.8609489185037456|
|[25.0,7.76,6.22,3...| 3.7162576320622343|
|[28.43,8.08,6.16,...| 3.8996569850747385|
|[30.2769999999999...|  3.732934703906416|
|[30.2769999999999...|  3.732934703906416|
|[30.2769999999999...| 3.2923510059219643|
|[33.0,4.9,5.6,2.4...|  3.024582674939758|
|[34.25,10.52,6.15...|  4.553096645773663|
|[38.0,10.56,5.67,...|  4.393234599056582|
|[42.0,15.04,7.08,...| 6.3530410488774125|
|[44.348,12.0,7.54...|  5.599301447538489|
|[45.0,11.78,7.54,...|  5.272247976411751|
|[46.0,7.0,6.7,1.8...| 2.5528247738127154|
|[47.262,14.86,7.2...|   6.27470220022536|
+----------

In [66]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lr_model.coefficients,lr_model.intercept))

Coefficients: [0.00616628676391,-0.191917258715,0.43694847031,0.988102713171,0.00428780109718,0.0447227626922] Intercept: -1.84228433239


In [67]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))
print("R2: {}".format(test_results.r2))

RMSE: 0.696299939167
MSE: 0.484833605285
R2: 0.961065004164
